In [1]:
!pip install pymongo

     |████████████████████████████████| 471 kB 855 kB/s eta 0:00:01


In [65]:
from pymongo import MongoClient
import pymongo

In [62]:
# Запускаем сервер MongoDB в Docker'e

# https://hub.docker.com/_/mongo
# docker pull mongo
# docker run --name some-mongo -d mongo:tag

# Подключаемся к серверу. IP адрес находим через docker inspect
# docker inspect {container_id} -> monogodb IP Address
client = MongoClient('172.17.0.2', 27017)
client

MongoClient(host=['172.17.0.2:27017'], document_class=dict, tz_aware=False, connect=True)

In [63]:
# Переключаемся на БД вакансий
db = client['vacancy']
# Создаем раздел вакансий с hh.ru
hh_vac = db.hh

In [57]:
# Считываем собраные вакансии из файла
import pandas
df_hh = pandas.read_csv('HeadHunter.csv')
df_hh = df_hh.drop(columns = ['Unnamed: 0'])
df_hh.head(2)

,min_salary,max_salary,currency,title,link,source
0,80000,0,руб,Учитель иностранного языка в школу,https://nn.hh.ru/vacancy/67622589?from=vacancy...,hh.ru
1,120000,0,руб,Учитель начальных классов,https://nn.hh.ru/vacancy/52238064?from=vacancy...,hh.ru


In [58]:
# Конвертируем в список словарей
d_hh = df_hh.to_dict('records')
d_hh[0]

{'min_salary': 80000,
 'max_salary': 0,
 'currency': 'руб',
 'title': 'Учитель иностранного языка в школу',
 'link': 'https://nn.hh.ru/vacancy/67622589?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D1%83%D1%87%D0%B8%D1%82%D0%B5%D0%BB%D1%8C',
 'source': 'hh.ru'}

In [59]:
# для кадой записи генерируем хеш

for id, item in enumerate(d_hh):
    d_hh[id]['hash'] = hash(frozenset(item.items()))
d_hh[0]

In [66]:
# создаем индекс в БД
hh_vac.create_index([('hash', pymongo.TEXT)], name='hash_index', unique=True)

'hash_index'

In [68]:
# загружаем первые 5 записей в БД

import pymongo

for i in range(5):
    doc = d_hh[i]
    # print(doc)
    hh_vac.insert_one(doc)

In [72]:
# смотрим кол-во записей в БД
from pprint import pprint
items_in_db = []
for item in hh_vac.find({}):
    items_in_db.append(item)
    # pprint(item)
len(items_in_db)

5

In [75]:
# загружаем первые 7 записей в БД

import pymongo
from pymongo import errors
for i in range(7):
    doc = d_hh[i]
    # print(doc)
    try:
        hh_vac.insert_one(doc)
    except errors.DuplicateKeyError:
        print(f"Document with hash = {doc['hash']} is already exists")
        

Document with id = -5985582358543763169 is already exists
Document with id = -2286417192751374999 is already exists
Document with id = 1950246790670858142 is already exists
Document with id = 7225811368836930956 is already exists
Document with id = -5045619242485330628 is already exists


In [76]:
# смотрим новое кол-во записей в БД
from pprint import pprint
items_in_db = []
for item in hh_vac.find({}):
    items_in_db.append(item)
    # pprint(item)
len(items_in_db)

7

In [77]:
# Загружаем все данные в БД

import pymongo
from pymongo import errors
for i in range(len(d_hh)):
    doc = d_hh[i]
    # print(doc)
    try:
        hh_vac.insert_one(doc)
    except errors.DuplicateKeyError:
        print(f"Document with hash = {doc['hash']} is already exists")

Document with hash = -5985582358543763169 is already exists
Document with hash = -2286417192751374999 is already exists
Document with hash = 1950246790670858142 is already exists
Document with hash = 7225811368836930956 is already exists
Document with hash = -5045619242485330628 is already exists
Document with hash = -5070998086419472134 is already exists
Document with hash = -2238314452646932468 is already exists


In [88]:
# Ищем вакансии с заработной платой больше введённой суммы 

salary_level = 90000

myquery = { "min_salary": { "$gt":salary_level }, "max_salary": { "$gt": salary_level } }
for item in hh_vac.find(myquery):
    pprint(item)

{'_id': ObjectId('62cc296f6381e422dbe47509'),
 'currency': 'руб',
 'hash': 1950246790670858142,
 'link': 'https://nn.hh.ru/vacancy/67628467?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D1%83%D1%87%D0%B8%D1%82%D0%B5%D0%BB%D1%8C',
 'max_salary': 105000,
 'min_salary': 95000,
 'source': 'hh.ru',
 'title': 'Учитель русского языка и литературы'}
{'_id': ObjectId('62cc2b0d6381e422dbe47512'),
 'currency': 'руб',
 'hash': -6468384864629407106,
 'link': 'https://nn.hh.ru/vacancy/67070468?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D1%83%D1%87%D0%B8%D1%82%D0%B5%D0%BB%D1%8C',
 'max_salary': 120000,
 'min_salary': 100000,
 'source': 'hh.ru',
 'title': 'Учитель начальной школы'}
{'_id': ObjectId('62cc2b0d6381e422dbe47513'),
 'currency': 'руб',
 'hash': 7174984906107890259,
 'link': 'https://nn.hh.ru/vacancy/67159514?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D1%83%D1%87%D0%B8%D1%82%D0%B5%D0%BB%D1%8C',
 'max_salary': 150000,
 'min_salary': 100000,
 